In [1]:
import gluonts
import nnts
import nnts.data
import nnts.plotting
import nnts.torch.preprocessing
import nnts.torch.models
import nnts.metrics
import nnts.torch.datasets
import nnts.loggers
import nnts.datasets
from nnts import utils
import gluonts.time_feature
import nnts.torch.utils
import nnts.torch.trainers
import nnts.metrics

import torch
torch.set_printoptions(precision=8, sci_mode=False)
%load_ext autoreload
%autoreload 2

### Load the dataset 
You can easily load any of the Monash datasets using the `load_dataset` function.

In [2]:
df, metadata = nnts.datasets.load_dataset("tourism_monthly")

### Set the Hyperparamters 
You will need to provide an optimizer and a loss function. Other hyperparameters are optional.

In [3]:
params = utils.Hyperparams(
    optimizer=torch.optim.Adam,
    loss_fn=nnts.torch.models.deepar.distr_nll,
    batch_size=32,
    batches_per_epoch=50,
    training_method=utils.TrainingMethod.TEACHER_FORCING,
    model_file_path="logs"
)

### Create covariate features
When using a DeepAR model we recommend using lag features as covariates.

In [4]:
lag_seq = gluonts.time_feature.lag.get_lags_for_frequency(metadata.freq)
lag_seq = [lag - 1 for lag in lag_seq if lag > 1]
lag_processor = nnts.torch.preprocessing.LagProcessor(lag_seq)

### Data Processing
We specify how we want to split the data, what transformations we want to apply and how we want to batch and sample the data during training.  

In [5]:
context_length = metadata.context_length + max(lag_seq)
dataset_options = {
    "context_length": metadata.context_length,
    "prediction_length": metadata.prediction_length,
    "conts": [],
    "lag_seq": lag_seq,
}

trn_dl, test_dl = nnts.torch.utils.create_dataloaders(
    df,
    nnts.datasets.split_test_train_last_horizon,
    context_length,
    metadata.prediction_length,
    Dataset=nnts.torch.datasets.TimeseriesLagsDataset,
    dataset_options=dataset_options,
    Sampler=nnts.torch.datasets.TimeSeriesSampler,
)

### Create the model
Create a Pytorch model.

In [6]:

net = nnts.torch.models.DistrDeepAR(
    nnts.torch.models.deepar.StudentTHead,
    params,
    nnts.torch.preprocessing.masked_mean_abs_scaling,
    1,
    lag_processor=lag_processor,
    scaled_features=[],
    context_length=metadata.context_length,
    cat_idx=None,
    seq_cat_idx=None,
)


### Train the model

In [7]:

trner = nnts.torch.trainers.TorchEpochTrainer(
    net,
    params,
    metadata
)
evaluator = trner.train(trn_dl)

DistrDeepAR(
  (decoder): UnrolledLSTMDecoder(
    (rnn): LSTM(16, 40, num_layers=2, batch_first=True, dropout=0.1)
  )
  (distribution): StudentTHead(
    (main): ModuleList(
      (0-2): 3 x Linear(in_features=40, out_features=1, bias=True)
    )
  )
)
Epoch 1 Train Loss: 9.322052955627441
Epoch 2 Train Loss: 8.89340591430664
Epoch 3 Train Loss: 8.534445762634277
Epoch 4 Train Loss: 8.416116714477539
Epoch 5 Train Loss: 8.37347412109375
Epoch 6 Train Loss: 8.330648422241211
Epoch 7 Train Loss: 8.18918228149414
Epoch 8 Train Loss: 8.084997177124023
Epoch 9 Train Loss: 7.9277262687683105
Epoch 10 Train Loss: 7.9218220710754395
Epoch 11 Train Loss: 7.872133255004883
Epoch 12 Train Loss: 7.820730686187744
Epoch 13 Train Loss: 7.812788486480713
Epoch 14 Train Loss: 7.835484504699707
Epoch 15 Train Loss: 7.794387340545654
Epoch 16 Train Loss: 7.780262470245361
Epoch 17 Train Loss: 7.789158821105957
Epoch 18 Train Loss: 7.8083930015563965
Epoch 19 Train Loss: 7.778183460235596
Epoch 20 Trai

### Evaluate

In [11]:
y_hat, y = evaluator.evaluate(
    test_dl, metadata.prediction_length, metadata.context_length
)

test_metrics = nnts.metrics.calc_metrics(
    y_hat, y, nnts.metrics.calculate_seasonal_error(trn_dl, metadata.seasonality)
)
test_metrics

using validate
using validate
using validate
using validate
using validate
using validate
using validate
using validate
using validate
using validate
using validate
using validate


{'mse': 51565356.0,
 'abs_error': 16258809.0,
 'abs_target_sum': 166958480.0,
 'abs_target_mean': 19007.11328125,
 'mase': 1.4145256280899048,
 'mape': 0.20941706001758575,
 'smape': 0.1823791265487671,
 'nd': 0.17429526150226593,
 'mae': 1850.957275390625,
 'rmse': 2336.250244140625,
 'seasonal_error': 1543.2191162109375}

In [12]:
nnts.plotting.plotly_forecasts_vs_actuals(y, y_hat)

### Forecast

In [13]:
net = evaluator.net

forecaster = nnts.torch.trainers.TorchForecaster(net)

In [14]:
forecast = forecaster.forecast(test_dl, metadata.prediction_length, metadata.context_length)

In [15]:
forecast

tensor([[[  6565.15136719],
         [  4102.58349609],
         [  2996.45263672],
         ...,
         [  3457.47680664],
         [  3948.36816406],
         [  6723.05175781]],

        [[192576.25000000],
         [155978.62500000],
         [150354.95312500],
         ...,
         [126516.64062500],
         [132509.46875000],
         [186100.56250000]],

        [[ 90530.55468750],
         [ 82137.88281250],
         [ 99747.16406250],
         ...,
         [133543.28125000],
         [123943.89843750],
         [106506.42968750]],

        ...,

        [[  7674.57763672],
         [ 14009.45117188],
         [ 10497.66113281],
         ...,
         [  7117.34912109],
         [  6616.79394531],
         [  7994.39111328]],

        [[  8145.37792969],
         [  7165.71826172],
         [ 11951.05175781],
         ...,
         [ 14460.15039062],
         [ 10635.84960938],
         [ 10441.82617188]],

        [[  2100.12548828],
         [  1699.74914551],
         [